In [1]:
import pandas as pd
import numpy as np

In [2]:
BASIC_PURCHASE_INFO = {
    'property_value':  650000,
    'purchase_price':  650000
}

In [3]:
MORTGAGE_INFO = {
    'interest_rate': 0.031, # %
    'down_payment': 0.25, # %
    'loan_term': 30
}

In [4]:
# initialize variables

BASIC_PURCHASE_INFO['down_payment'] = BASIC_PURCHASE_INFO['purchase_price'] * MORTGAGE_INFO['down_payment']
BASIC_PURCHASE_INFO['equity_at_purchase'] = BASIC_PURCHASE_INFO['property_value'] - BASIC_PURCHASE_INFO['purchase_price'] + BASIC_PURCHASE_INFO['down_payment']

MORTGAGE_INFO['loan_amount'] = BASIC_PURCHASE_INFO['purchase_price'] * (1 - MORTGAGE_INFO['down_payment'])
MORTGAGE_INFO['monthly_payment'] = 2081.70 # hardcoded for now
MORTGAGE_INFO['annual_payment'] = MORTGAGE_INFO['monthly_payment'] *12

In [5]:
CALC_ASSUM = {
    'rental_income': 2700,
    'vacancy': 0,
    'maintenance_reserve': 0,
    'management_fee': 0.15, # %
    'monthly_property_tax': 167.00,
    'monthly_insurance': 50.00,
    'utilities': {
        'monthly_electricity': 30,
        'monthly_gas': 30,
        'monthly_water': 30
    },
    'rental_income_increase': 2, #%
    'property_tax_increase': 2,  #%
    'closing_cost_buy': 0.02, #%
    'closing_cost_sell': 0
}

BASIC_PURCHASE_INFO['total_cash_required'] = BASIC_PURCHASE_INFO['down_payment'] + CALC_ASSUM['closing_cost_buy']*MORTGAGE_INFO['loan_amount']

In [6]:
BASIC_PURCHASE_INFO['total_cash_required']

172250.0

In [7]:
def gross_scheduled_income(year):
    return round(12*CALC_ASSUM['rental_income'] * (1+CALC_ASSUM['rental_income_increase']/100)**((year * 12 - 12 ) / 12), 2)

In [8]:
def property_tax(year):
    return round(12*CALC_ASSUM['monthly_property_tax']*(1+CALC_ASSUM['property_tax_increase']/100) ** ((year * 12 - 12 ) / 12), 2)

In [9]:
def generate_years():
    return range(1,31)

In [10]:
# calculating income

gross_income = np.array([gross_scheduled_income(i) for i in generate_years()])

total_operating_income = gross_income + 0

In [11]:
# calculate operating expenses

property_taxes = np.array([property_tax(i) for i in generate_years()])
insurance = np.full(max(generate_years()), CALC_ASSUM['monthly_insurance']*12)

# calculate total utilities
total_util = 0
for util in CALC_ASSUM['utilities'].keys():
    total_util += CALC_ASSUM['utilities'][util]

utility = np.full(max(generate_years()), total_util*12)
property_management = gross_income * CALC_ASSUM['management_fee'] 

total_operating_expense = property_taxes + insurance + utility + property_management

In [12]:
property_management

array([4860.    , 4957.2   , 5056.344 , 5157.471 , 5260.62  , 5365.833 ,
       5473.149 , 5582.613 , 5694.264 , 5808.15  , 5924.313 , 6042.7995,
       6163.6545, 6286.9275, 6412.6665, 6540.9195, 6671.739 , 6805.173 ,
       6941.277 , 7080.102 , 7221.705 , 7366.1385, 7513.461 , 7663.731 ,
       7817.0055, 7973.3445, 8132.8125, 8295.468 , 8461.377 , 8630.6055])

In [13]:
total_operating_expense

array([ 8544.    ,  8681.28  ,  8821.304 ,  8964.131 ,  9109.81  ,
        9258.413 ,  9409.979 ,  9564.583 ,  9722.274 ,  9883.12  ,
       10047.173 , 10214.5195, 10385.2145, 10559.3175, 10736.9065,
       10918.0395, 11102.799 , 11291.253 , 11483.487 , 11679.552 ,
       11879.545 , 12083.5385, 12291.601 , 12503.841 , 12720.3155,
       12941.1145, 13166.3425, 13396.068 , 13630.387 , 13869.3955])

In [14]:
net_operating_income = total_operating_income - total_operating_expense

In [15]:
annual_cash_flow = net_operating_income - MORTGAGE_INFO['annual_payment']
monthly_cash_flow = annual_cash_flow / 12

In [16]:
# calculate financial performance

In [17]:
cap_rate = net_operating_income / BASIC_PURCHASE_INFO['property_value']

In [18]:
cash_on_cash = np.round((annual_cash_flow / BASIC_PURCHASE_INFO['total_cash_required'])*100, 2)

In [19]:
cash_on_cash

array([-0.65, -0.36, -0.05,  0.25,  0.57,  0.89,  1.22,  1.55,  1.89,
        2.24,  2.59,  2.96,  3.32,  3.7 ,  4.08,  4.47,  4.87,  5.28,
        5.7 ,  6.12,  6.55,  6.99,  7.44,  7.9 ,  8.37,  8.84,  9.33,
        9.83, 10.33, 10.85])

In [20]:
-np.pmt(MORTGAGE_INFO['interest_rate'] / 12, MORTGAGE_INFO['loan_term']*12, MORTGAGE_INFO['loan_amount'])

2081.704944660372

In [21]:
def cumulative_principal(start, end, rate, nper, pv):

    cp = 0
    for i in range(start,end+1):
        cp = cp + (-np.ppmt(rate / 12, i, nper*12, pv))
    
    return cp

In [22]:
principal_paydown = np.array([cumulative_principal(i*12-11, i*12, MORTGAGE_INFO['interest_rate'], MORTGAGE_INFO['loan_term'], MORTGAGE_INFO['loan_amount']) for i in generate_years()])

In [23]:
mortgage_balance = np.array([round(np.fv(MORTGAGE_INFO['interest_rate'] / 12, i*12, 2081.7, -MORTGAGE_INFO['loan_amount']), 2) for i in generate_years()])

In [24]:
mortgage_balance

array([4.7749068e+05, 4.6716662e+05, 4.5651793e+05, 4.4553440e+05,
       4.3420551e+05, 4.2252038e+05, 4.1046782e+05, 3.9803628e+05,
       3.8521384e+05, 3.7198820e+05, 3.5834670e+05, 3.4427624e+05,
       3.2976336e+05, 3.1479412e+05, 2.9935419e+05, 2.8342876e+05,
       2.6700257e+05, 2.5005987e+05, 2.3258441e+05, 2.1455946e+05,
       1.9596772e+05, 1.7679138e+05, 1.5701205e+05, 1.3661077e+05,
       1.1556800e+05, 9.3863540e+04, 7.1476610e+04, 4.8385740e+04,
       2.4568790e+04, 2.9300000e+00])

In [25]:
equity_wealth = BASIC_PURCHASE_INFO['property_value'] - mortgage_balance

In [26]:
effective_net_cash_flow = annual_cash_flow + principal_paydown

In [27]:
return_on_equity = effective_net_cash_flow / equity_wealth

In [28]:
plus_sales_proceeds_if_final_year = annual_cash_flow + equity_wealth - BASIC_PURCHASE_INFO['property_value']*CALC_ASSUM['closing_cost_sell']

In [29]:
accumulated_cash_flow = np.cumsum(annual_cash_flow)

In [30]:
accumulated_cash_flow

array([ -1124.4   ,  -1738.08  ,  -1830.824 ,  -1392.215 ,   -411.625 ,
         1121.782 ,   3219.063 ,   5891.5   ,   9150.586 ,  13008.066 ,
        17475.913 ,  22566.3235,  28291.739 ,  34664.8715,  41698.675 ,
        49406.3655,  57801.4265,  66897.5935,  76708.8865,  87249.6145,
        98534.3695, 110578.021 , 123395.76  , 137003.059 , 151415.7135,
       166649.829 , 182721.8365, 199648.4885, 217446.8815, 236134.456 ])

In [31]:
annual_cash_flow

array([-1124.4   ,  -613.68  ,   -92.744 ,   438.609 ,   980.59  ,
        1533.407 ,  2097.281 ,  2672.437 ,  3259.086 ,  3857.48  ,
        4467.847 ,  5090.4105,  5725.4155,  6373.1325,  7033.8035,
        7707.6905,  8395.061 ,  9096.167 ,  9811.293 , 10540.728 ,
       11284.755 , 12043.6515, 12817.739 , 13607.299 , 14412.6545,
       15234.1155, 16072.0075, 16926.652 , 17798.393 , 18687.5745])

In [32]:
effective_future_value = plus_sales_proceeds_if_final_year + accumulated_cash_flow + annual_cash_flow

In [33]:
effective_future_value

array([169136.12  , 179867.94  , 191465.758 , 203950.603 , 217344.045 ,
       231668.216 , 246945.805 , 263200.094 , 280454.918 , 298734.826 ,
       318064.907 , 338470.9045, 359979.21  , 382617.0165, 406412.092 ,
       431392.9865, 457588.9785, 485030.0575, 513747.0625, 543771.6105,
       575136.1595, 607873.944 , 642019.188 , 677606.887 , 714673.0225,
       753254.52  , 793389.2415, 835116.0525, 878474.8775, 923506.675 ])

In [34]:
annualized_return = effective_future_value / BASIC_PURCHASE_INFO['total_cash_required']

In [35]:
annualized_return = effective_future_value / BASIC_PURCHASE_INFO['total_cash_required']

for i in range(0,len(annualized_return)):
    annualized_return[i] = annualized_return[i]**(1/(i+1))-1


In [36]:
annualized_return

array([-0.01807768,  0.0218738 ,  0.03588281,  0.04313722,  0.04760534,
        0.0506334 ,  0.05280737,  0.05442538,  0.05565655,  0.056605  ,
        0.05733895,  0.05790542,  0.05833821,  0.05866244,  0.05889726,
        0.05905768,  0.05915564,  0.0592008 ,  0.05920111,  0.05916316,
        0.05909246,  0.0589937 ,  0.05887084,  0.05872729,  0.05856596,
        0.0583894 ,  0.05819979,  0.05799903,  0.05778881,  0.05757058])

In [37]:
return_on_investment = (plus_sales_proceeds_if_final_year / BASIC_PURCHASE_INFO['total_cash_required']) / BASIC_PURCHASE_INFO['total_cash_required']

In [54]:
d = pd.DataFrame({
    'Year': [i for i in generate_years()],
    'Cash on Cash COC': cash_on_cash,
    'Return on Equity ROE': return_on_equity,
    'Annualized Return APY': annualized_return,
    'Return On Investment': return_on_investment*100
})

d.set_index('Year').T

Year,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
Cash on Cash COC,-0.650000,-0.360000,-0.050000,0.250000,0.570000,0.890000,1.220000,1.550000,1.890000,2.240000,...,6.550000,6.990000,7.440000,7.900000,8.370000,8.840000,9.330000,9.830000,10.330000,10.850000
Return on Equity ROE,0.051504,0.053111,0.054558,0.055864,0.057043,0.058109,0.059073,0.059945,0.060734,0.061448,...,0.065803,0.065975,0.066122,0.066243,0.066343,0.066420,0.066478,0.066517,0.066539,0.066544
Annualized Return APY,-0.018078,0.021874,0.035883,0.043137,0.047605,0.050633,0.052807,0.054425,0.055657,0.056605,...,0.059092,0.058994,0.058871,0.058727,0.058566,0.058389,0.058200,0.057999,0.057789,0.057571
Return On Investment,0.000578,0.000614,0.000652,0.000691,0.000731,0.000772,0.000814,0.000858,0.000903,0.000950,...,0.001568,0.001635,0.001705,0.001776,0.001850,0.001926,0.002004,0.002085,0.002168,0.002254


In [57]:
y = [i for i in generate_years()]

In [78]:
from operator import itemgetter 
list(range(1,11)) + [10, 20, 30]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 20, 30]